# Actualizacion de informacion de MyHandling

### 1ra etapa:
##### El bot descarga el excel de flight paths automaticamente

In [39]:
#%pip install selenium
#%pip install webdriver_manager

In [24]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [44]:
# Initiate the browser
browser  = webdriver.Chrome(ChromeDriverManager().install())

# Open the Website
browser.get('https://ap.myhandlingsoftware.com/Account/Login?ReturnUrl=%2f')

#Login info
myHandling_email = 'David.vallve@skyvalet.com'
myHandling_password = '88888888'

#Input password
browser.find_element(By.XPATH,'//*[@id="UserName"]').send_keys(myHandling_email)
browser.find_element(By.XPATH,'//*[@id="Password"]').send_keys(myHandling_password)
browser.find_element(By.XPATH,'/html/body/div[2]/div/div[5]/div/div[1]/div/div[1]/form/div[3]/div/input').click()

#Redirigo a Flight Paths y descargo el excel
browser.get('https://ap.myhandlingsoftware.com/Home/BoardList')
browser.find_element(By.XPATH,'//*[@id="ExportLinkExcel"]').click()

C:\Users\Pedro Vallarino\AppData\Local\Temp\ipykernel_10508\948112794.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser  = webdriver.Chrome(ChromeDriverManager().install())


### 2da etapa:
##### El bot sube esta informacion de la carpeta de descargas a el drive

In [86]:
#%pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
#%pip install Google

In [1]:
import pickle
import os
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google.auth.transport.requests import Request

In [ ]:
def Create_Service(client_secret_file, api_name, api_version, *scopes):
    print(client_secret_file, api_name, api_version, scopes, sep='-')
    CLIENT_SECRET_FILE = client_secret_file
    API_SERVICE_NAME = api_name
    API_VERSION = api_version
    SCOPES = [scope for scope in scopes[0]]
    print(SCOPES)

    cred = None

    pickle_file = f'token_{API_SERVICE_NAME}_{API_VERSION}.pickle'
    # print(pickle_file)

    if os.path.exists(pickle_file):
        with open(pickle_file, 'rb') as token:
            cred = pickle.load(token)

    if not cred or not cred.valid:
        if cred and cred.expired and cred.refresh_token:
            cred.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            cred = flow.run_local_server()

        with open(pickle_file, 'wb') as token:
            pickle.dump(cred, token)

    try:
        service = build(API_SERVICE_NAME, API_VERSION, credentials=cred)
        print(API_SERVICE_NAME, 'service created successfully')
        return service
    except Exception as e:
        print('Unable to connect.')
        print(e)
        return None

def convert_to_RFC_datetime(year=1900, month=1, day=1, hour=0, minute=0):
    dt = datetime.datetime(year, month, day, hour, minute, 0).isoformat() + 'Z'
    return dt

: 

In [ ]:
from googleapiclient.http import MediaFileUpload
#from Google import Create_Service

: 

In [ ]:
CLIENT_SECRET_FILE = 'credentials.json'
API_NAME = 'drive'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/drive']

Service = Create_Service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)

folder_id = []
file_names = []
mime_types = []

for file_name, mime_type in zip(file_names, mime_types):
    file_metadata = {
        'name': file_name,
        'parents': [folder_id]
    }

    media = MediaFileUpload('./Random Files/{0}'.format(file_name), mimetype=mime_type )

    service.files().create(
        body = file_metadata,
        media_body = media,
        fields = 'id'
    ).execcute()


: 

: 